# Agent as a Tool: Dynamic Multi-Agent Workflows

## Overview

In AutoGen AgentChat, any `BaseChatAgent` can be wrapped and used as a tool by another agent. This powerful feature enables dynamic, model-driven multi-agent workflows, where a primary agent can decide to invoke other specialized agents to solve sub-tasks.

This notebook demonstrates how to use `autogen_agentchat.tools.AgentTool` to achieve this agent-as-a-tool pattern, fostering more complex and adaptive agentic behaviors.

## Prerequisites

Ensure you have the necessary packages installed:

In [1]:
!pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai]>=0.7" rich

# IMPORTANT: See: https://github.com/microsoft/autogen/issues/6906
!pip install --quiet --force-reinstall "openai==1.80"

> **⚠️ IMPORTANT:**  
> If you just ran the `!pip install --quiet --force-reinstall "openai==1.80"` command,  
> you **must** restart the Jupyter kernel before continuing.  
> This ensures the newly installed `openai` package is loaded into memory  
> and avoids mixed-version issues that can cause runtime errors.  
>  
> **In Jupyter:** go to **Kernel → Restart & Clear Output**, then rerun the notebook from the top.

## Example: A Summarizer Agent as a Tool

We will create a simple 'Summarizer Agent' whose sole purpose is to summarize text. Then, we will wrap this summarizer agent as a tool and provide it to a 'Primary Agent'. The primary agent will then decide when and how to use the summarizer tool based on its task.

In [2]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.tools import AgentTool
from autogen_agentchat.ui import Console

async def run_agent_as_tool_example():
    # 1. Initialize a model client
    model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")

    # 2. Create the Summarizer Agent (the agent that will become a tool)
    summarizer_agent = AssistantAgent(
        name="summarizer",
        model_client=model_client,
        system_message="You are a helpful assistant that summarizes text concisely."
    )

    # 3. Wrap the Summarizer Agent as an AgentTool
    # The description is crucial for the primary agent to understand when to use this tool.
    summarize_tool = AgentTool(
        summarizer_agent,
        # name="summarize_text",
        # description="Summarizes a given text concisely. Input: text (str). Output: summary (str)."
    )

    # 4. Create the Primary Agent and provide it with the AgentTool
    primary_agent = AssistantAgent(
        name="primary_assistant",
        model_client=model_client,
        tools=[summarize_tool], # The wrapped agent is now a tool
        system_message="You are a versatile assistant. Use the 'summarize_text' tool if the user asks for a summary."
    )

    # 5. Run a task with the primary agent
    long_text = (
        "The quick brown fox jumps over the lazy dog. This is a classic pangram, "
        "a sentence that contains every letter of the alphabet at least once. "
        "Pangrams are often used to display typefaces or to test the functionality of keyboards. "
        "Another famous pangram is 'Pack my box with five dozen liquor jugs.'"
    )

    task = f"Please summarize the following text: {long_text}"

    print("\n🔄 Task: What is my public IP address?")
    print("-" * 40)
    await Console(primary_agent.run_stream(task=task))

    # Clean up resources
    await model_client.close()

await run_agent_as_tool_example()


🔄 Task: What is my public IP address?
----------------------------------------
---------- TextMessage (user) ----------
Please summarize the following text: The quick brown fox jumps over the lazy dog. This is a classic pangram, a sentence that contains every letter of the alphabet at least once. Pangrams are often used to display typefaces or to test the functionality of keyboards. Another famous pangram is 'Pack my box with five dozen liquor jugs.'
---------- ToolCallRequestEvent (primary_assistant) ----------
[FunctionCall(id='call_EKI5zjuF7aXSuJpyuufQXp9d', arguments='{"task":"The quick brown fox jumps over the lazy dog is a classic pangram containing every letter of the alphabet. Pangrams are used to display typefaces and test keyboards. Another example is \'Pack my box with five dozen liquor jugs.\'"}', name='summarizer')]
---------- TextMessage (user) ----------
The quick brown fox jumps over the lazy dog is a classic pangram containing every letter of the alphabet. Pangrams ar

## How it Works

When the `primary_agent` receives the task, its internal LLM analyzes the request. Because the `summarize_text` tool (which is actually our `summarizer_agent`) has a clear description indicating it can summarize text, the LLM decides to call this tool. The `AgentTool` then orchestrates the execution of the `summarizer_agent` with the provided text as its input. The output of the `summarizer_agent` is then returned to the `primary_agent` as the tool's result, which the `primary_agent` can then present as its response.

This pattern allows for:

- **Modularity**: Break down complex tasks into smaller, specialized agents.
- **Reusability**: Reuse agents as tools across different workflows.
- **Dynamic Dispatch**: The LLM dynamically decides which agent-tool to use based on the task context.
- **Hierarchical Reasoning**: Create layers of agents, where higher-level agents delegate to lower-level specialized agents.

## Next Steps

Experiment with the agent-as-a-tool pattern:

1.  **Create other specialized agents**: For example, a 'Code Generator Agent' or a 'Data Analysis Agent' and use them as tools.
2.  **Chain agent tools**: Have one agent-tool call another agent-tool for multi-step delegation.
3.  **Explore multi-agent conversations**: Combine this with multi-agent conversations where agents can directly chat and also use each other as tools.